In [45]:
import csv
from collections import defaultdict

organism = defaultdict(lambda: defaultdict(list))
summary = defaultdict(lambda: defaultdict(int))
with open('test.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        org = row['Organism_Name'].split(" (")[0]
        organism[org][row["Nuc_Completeness"]].append(row['Accession'])
        for key in ['Nuc_Completeness', 'Geo_Location', 'Species', 'Genus', 'Family']:
            summary[key][row[key]] += 1

In [82]:
import subprocess
import random
import gzip
import json


def download_accession(org_name, ncbi_accessions, directory, threads=4):
    try:
        cmd = ["ncbi-acc-download", "--format", "fasta", "--out", "/dev/stdout"]
        cmd.extend(ncbi_accessions)
        print(f"\nRunning: {' '.join(cmd)}")
        proc1 = subprocess.run(cmd, stdout=subprocess.PIPE, check=True)
        
        cmd = ["grep", "."]
        #print(f"\nRunning: {' '.join(cmd)}")
        proc2 = subprocess.run(cmd, input=proc1.stdout, stdout=subprocess.PIPE, check=True)
    
        cmd = ["bgzip"]
        #print(f"\nRunning: {' '.join(cmd)}")
        handle = open(f"{directory}/{org_name.replace(' ','_').replace('/','_')}.fa.gz", "w")
        proc3 = subprocess.run(cmd, input=proc2.stdout, stdout=handle, check=True)
        handle.close()
        return True
    except:
        print("FAILED, skip")
        return False

In [85]:
import os

def download(organism, max_per_organism = 10):
    os.makedirs("refs", exist_ok=True)
    downloaded = defaultdict(list)
    for key in organism:
        acc_list = random.sample(organism[key]["complete"],min(max_per_organism, len(organism[key]["complete"])))
        if len(acc_list) < max_per_organism:
            acc_list.extend(random.sample(organism[key]["partial"],min(max_per_organism-len(acc_list), len(organism[key]["partial"]))))
        status = download_accession(key, acc_list, "refs")
        if status:
            downloaded[key] = acc_list
    return downloaded
    

In [ ]:
downloaded = download(organism, 10)

with open("refs.json", "w") as outfile: 
    json.dump(downloaded, outfile)


Running: ncbi-acc-download --format fasta --out /dev/stdout EU256104 EU781748 EU155252 EU256017 EU256012 EU256070 EU155287 EU155338 EU482884 EU155251

Running: ncbi-acc-download --format fasta --out /dev/stdout BK058042 BK053466 BK036820 BK059043 BK049315 BK055962 BK045896 BK033190 BK044629 BK047080

Running: ncbi-acc-download --format fasta --out /dev/stdout AB499693 AB731741 KX174313 J04498 EU980602 X61240 KY025541 KX174312 AB731740 AB731742

Running: ncbi-acc-download --format fasta --out /dev/stdout JQ431254 KX595339 KX697013 KJ942787 KX078496 JQ173163 KJ942668 KY435816 JQ431338 KJ942798
FAILED, skip

Running: ncbi-acc-download --format fasta --out /dev/stdout MN090888 ON902514 ON902754 KX232610 MT942827 OK532559 AY835767 EF637051 MT191012 MN090529

Running: ncbi-acc-download --format fasta --out /dev/stdout MW384273 MW384142 MW384280 MW383985 KF371760 MW384111 MW384137 KF726063 KF371816 KF371695

Running: ncbi-acc-download --format fasta --out /dev/stdout MK139018 OM513934 OM5139

In [ ]:
import subprocess

def get_download_accession(ncbi_accession):
    # Execute the command (prefetching) and capture its output
    entrez_cmd = ["esearch", "-db", "nuccore", "-query", ncbi_accession, "| elink -target assembly | esummary | xtract -pattern DocumentSummary -element Genbank"]
    print(f"\nRunning: {' '.join(entrez_cmd)}")
    entrez = subprocess.Popen(entrez_cmd, stdout=subprocess.PIPE)
    output, _ = entrez.communicate()

    # Convert the output bytes to a string
    output = output.decode('utf-8')
    print(output)


In [5]:
import subprocess

def get_download_accession(ncbi_accession):
    # Execute the command (prefetching) and capture its output
    entrez_cmd = ["esearch", "-db", "nuccore", "-query", ncbi_accession, "| elink -target assembly | esummary | xtract -pattern DocumentSummary -element Genbank"]
    print(f"\nRunning: {' '.join(entrez_cmd)}")
    entrez = subprocess.Popen(entrez_cmd, stdout=subprocess.PIPE)
    output, _ = entrez.communicate()

    # Convert the output bytes to a string
    output = output.decode('utf-8')
    print(output)


In [6]:
for acc in organism['Mammalian orthoreovirus']:
    print(acc)
    get_download_accession(acc)
    break

MG999576

Running: esearch -db nuccore -query MG999576 | elink -target assembly | esummary | xtract -pattern DocumentSummary -element Genbank


 ERROR:  Unrecognized argument | elink -target assembly | esummary | xtract -pattern DocumentSummary -element Genbank


<ENTREZ_DIRECT>
  <Db>nuccore</Db>
  <WebEnv>MCID_66ab77efa7e10864201aa2e2</WebEnv>
  <QueryKey>1</QueryKey>
  <Count>1</Count>
  <Step>1</Step>
  <Elapsed>1</Elapsed>
</ENTREZ_DIRECT>

